# ECE 5831 driving LLM

## Dataset mounting and downloading

In [1]:
from google.colab import drive


drive.mount('/content/drive')

file_id1 = '1TyinR8tm9sRPItzFzKpVktOv9Vu9v-HM'
file_id2 = '1jtOLDFR0YQc2IPVRvdUDZogjNTjIkvg_'

file_url1 = f'https://drive.google.com/uc?id={file_id1}'
file_url2 = f'https://drive.google.com/uc?id={file_id2}'

!gdown {file_url1}
!gdown {file_url2}


Mounted at /content/drive
Downloading...
From: https://drive.google.com/uc?id=1TyinR8tm9sRPItzFzKpVktOv9Vu9v-HM
To: /content/vqa_train_10k.tar.gz
100% 21.7M/21.7M [00:00<00:00, 96.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1jtOLDFR0YQc2IPVRvdUDZogjNTjIkvg_
To: /content/vqa_test_1k.tar.gz
100% 2.07M/2.07M [00:00<00:00, 201MB/s]


## Installing libraries

In [2]:
!pip install bitsandbytes==0.37.2 \
  datasets==2.10.1 \
  einops==0.4.1 \
  fire==0.5.0 \
  gradio==3.34.0 \
  openai==0.27.0 \
  peft==0.2.0 \
  pillow==9.3.0 \
  retry==0.9.2 \
  sentencepiece==0.1.99 \
  transformers==4.30.0 \
  triton==2.0.0 \
  tqdm==4.64.0 \
  wandb==0.15.4 \
  click==8.1.6


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 106.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 124.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 99.5 MB/s eta 0

## Installing wandb for displaying results

In [3]:
!pip install wandb

In [4]:
import os
os.environ["WANDB_API_KEY"] = "6b6f1fe1c4604bd06af83041c1e3486d311b3342"


## Imported wandb with API key

In [5]:
import wandb

#  W&B API key
wandb.login(key="6b6f1fe1c4604bd06af83041c1e3486d311b3342")

# Initialize  project
wandb.init(project="driving_llm")


wandb: Currently logged in as: anubhuti. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: wandb version 0.16.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in /content/wandb/run-20231209_205722-ycoa8kcq
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run usual-planet-6
wandb: ⭐️ View project at https://wandb.ai/anubhuti/driving_llm
wandb: 🚀 View run at https://wandb.ai/anubhuti/driving_llm/runs/ycoa8kcq


In [6]:
import os

# Create the destination directory if it doesn't exist
destination_directory = '/content/extracted_data/'
os.makedirs(destination_directory, exist_ok=True)

# Extract the test data
!tar -xzvf /content/vqa_test_1k.tar.gz -C $destination_directory

# Extract the train data
!tar -xzvf /content/vqa_train_10k.tar.gz -C $destination_directory

vqa_test_1k.pkl
vqa_train_10k.pkl


## Loading the data file

In [7]:
import pickle

# Open the .pkl file in binary mode (rb: read binary)
with open('/content/extracted_data/vqa_train_10k.pkl', 'rb') as file:
    # Load the data from the file
    data = pickle.load(file)

with open('/content/extracted_data/vqa_test_1k.pkl', 'rb') as file:
    # Load the data from the file
    data_test = pickle.load(file)

# Now 'data' contains the contents of the .pkl file
print(data[:5])

[{'frame_num': 0, 'observation': {'ego_vehicle_descriptor': tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.3250, 1.0050, 0.7800, 1.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.3679, 0.0000,
        0.0000, 0.0798, 0.0798, 0.0798, 0.1423, 0.1751, 1.0000, 1.0000, 0.8573,
        0.4652, 0.3694, 0.3694, 0.4039]), 'liable_vehicles': None, 'pedestrian_descriptors': tensor([[ 1.0000,  1.7488,  3.0088,  0.3324,  0.1082,  0.9687,  0.2483,  0.0000,
          0.0000],
        [ 1.0000,  1.5271,  4.8531, -0.6271,  0.1032, -1.0000, -0.0086,  0.0000,
          0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000],
       

In [8]:
import random
t_try_data = random.sample(data, 100)
t_try_test_data = random.sample(data_test, 10)

# Save the randomly chosen entries to a new pickle file
with open('t_try_data.pkl', 'wb') as file:
    pickle.dump(t_try_data, file)

with open('t_try_test_data.pkl', 'wb') as file:
    pickle.dump(t_try_test_data, file)

In [10]:
!git clone https://github.com/wayveai/Driving-with-LLMs.git

Cloning into 'Driving-with-LLMs'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 104 (delta 6), reused 12 (delta 5), pack-reused 90
Receiving objects: 100% (104/104), 317.11 MiB | 27.56 MiB/s, done.
Resolving deltas: 100% (32/32), done.
Updating files: 100% (34/34), done.


## Training the model

In [30]:
!python train.py \
    --mode train \
    --eval_steps 50 \
    --val_set_size 32 \
    --num_epochs 5 \
    --resume_from_checkpoint models/weights/stage1_pretrained_model/ \
    --data_path ./t_try_data.pkl \
    --val_data_path ./t_try_test_data.pkl \
    --vqa


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/ma

## Checking available CUDA memory

In [19]:
!nvidia-smi

Fri Dec  8 23:04:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
import os
os.environ['WANDB_DISABLED'] = 'false'

## Evaluating the model

In [ ]:
!python train.py \
    --mode eval \
    --resume_from_checkpoint models/weights/stage2_with_pretrained/ \
    --data_path ./t_try_data.pkl \
    --val_data_path ./t_try_test_data.pkl \
    --eval_items caption,action \
    --vqa


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('8013'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/ma